In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
#%tensorboard --logdir runs


In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

# Constants
DATA_PATH = "../../data/iris_binary.csv"




In [ ]:
def read_data(dataPath):
    data = pd.read_csv(DATA_PATH)
    print(data.head())
    return torch.tensor(data.values,dtype=torch.float)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
class FirstModel(nn.Module):
    def __init__(self):
        super(FirstModel, self).__init__()
        # first layer: input layer (4 values)
        self.fc1 = nn.Linear(4,4)
        # second layer: output layer layer (4 input values, 1 output layer)
        self.fc2 = nn.Linear(4,1)

    # function for forwarding: must be passed through all layers
    def forward(self, x):

        # input layer - activation:
        #x = torch.sigmoid(self.fc1(x))
        x = self.fc1(x)

        # output layer - activation:
        x = torch.sigmoid(self.fc2(x))

        #print("forwar: return ", x)
        return x


In [ ]:
#nn.Linear?


In [ ]:
# read iris data
dataArr = read_data(DATA_PATH)
# print("dataArr: ", dataArr)

# create writer for tensorboard
now = datetime.now().strftime('%Y%m%d-%H:%M:%S')
print(now)
#print(type(now))
outPath = 'runs/02-pytorch/' + now
#writer = SummaryWriter('runs/02-pytorch')
writer = SummaryWriter(outPath)

# create Model
model = FirstModel()

# select appropriate loss function (BCE loss fits in this case: range 0:1)
loss = nn.BCELoss()

# select optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# use high number for epochs to test tensorboard
numEpochs = 1000

errArr = np.zeros([numEpochs])
# print("errors, shape: ", errArr.shape)
# print("errors, ndim: ", errArr.ndim)

for epoch in range(numEpochs):
    # reset optimizer
    optimizer.zero_grad()

    # select data
    X = dataArr[:, 0:4]
    # print("X: ", X)
    y = dataArr[:, 4]
    y = y.view(y.shape[0], 1)

    modelOut = model(X)

    # print("Model:\n", modelOut)
    # print("model dictionary: \n", model.state_dict())
    # print("Shape(y): ", y.shape)
    error = loss(modelOut, y)

    # visualize progress in Tensorboard
    writer.add_scalar("Loss over epochs", error.item(), epoch)

    # print("Error: ", error)

    # back propagation to improve weights
    error.backward()

    # optimize weights and bias
    optimizer.step()

    # store error and weight for visualization later
    errArr[epoch] = error


In [ ]:
# +++ Visualization +++
plt.plot(errArr)
plt.title("Error over epochs")
plt.xlabel("Number of epochs")
plt.ylabel("Error")
